
This Jupiter notebook is used to download, pre-process and save data about all test data in the Czech Republic from the webpage of the Czech...... Output from this notebook is CSV file with downloaded data about........ that will be further processed, cleaned and aggregated.

os package is used only to check whether we are in the correct working directory.

In the def, we will be applying methods from requests (downloading site content), BeautifulSoup (machine reading of downloaded data), tqdm (interactive measurement of the downloading progress) and time (setting pause between requests from the site) packages.

After the data are downloaded we use pandas to pre-process and save them.

After downloading the packages, I will check the data structure in order to make sure that we can scrape the given information here below we used BeutfulSoup library to request the data.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from tqdm import tqdm
import re
import numpy as np

In [2]:
html_text = requests.get("https://testovani.uzis.cz/").text.strip()
soup = BeautifulSoup(html_text, 'lxml')

In [3]:
# For loop finding all ancor elements:
for a in soup.find_all('a', class_ = "list__link", href=True):
    print ("Found the URL:", a['href'])

Found the URL: /Detail?id=a81a99e2-90bd-494c-ae42-7090f93115ca&backURL=/
Found the URL: /Detail?id=42c85d23-21e4-4fe9-baef-9ab06c0cbe6d&backURL=/
Found the URL: /Detail?id=4bf34117-aadf-400d-91ec-77dc8633874f&backURL=/
Found the URL: /Detail?id=e3bc023c-27a1-496f-8a5c-665b0e103e60&backURL=/
Found the URL: /Detail?id=4e6f8c5e-1b70-4289-a2ab-ae9065304eb5&backURL=/
Found the URL: /Detail?id=5fe2de84-e229-4505-8cc3-4a7d11035228&backURL=/
Found the URL: /Detail?id=3960f5a9-8269-487c-8b93-f1fee91346ce&backURL=/
Found the URL: /Detail?id=c8986ef2-a361-4830-9fc2-c301cae6dc2a&backURL=/
Found the URL: /Detail?id=755322f8-8f3d-4048-94d6-d01a39701638&backURL=/
Found the URL: /Detail?id=3c58898a-034a-4056-80c6-d3f14748f54c&backURL=/
Found the URL: /Detail?id=2f2efdc8-63ea-4ab1-b6cd-9f39896ee2d8&backURL=/
Found the URL: /Detail?id=8c49206d-ff94-4183-a1eb-df190d9f345b&backURL=/
Found the URL: /Detail?id=0d6a6d59-1ae7-43d8-8dc1-a8ecbb8a425c&backURL=/
Found the URL: /Detail?id=5ecf72d5-b142-4aa6-aeef-0

In [4]:
# Getting an unique identifier of each place - the id seems to be stored in href of <a> tag right after "?id="
# All obtained Ids are then assigned to TestCenterIds array

testingCenterHyperlinkTags = soup.find_all('a', class_ = "list__link", href=True)

TestCenterIds = []

for testingCenterHyperlinkTag in testingCenterHyperlinkTags[:5]:
    testingCenterHyperlink = testingCenterHyperlinkTag['href'][11:47]
    print ("Found the id:", testingCenterHyperlink)
    TestCenterIds.append(testingCenterHyperlink)

TestCenterIds[0:5]

Found the id: a81a99e2-90bd-494c-ae42-7090f93115ca
Found the id: 42c85d23-21e4-4fe9-baef-9ab06c0cbe6d
Found the id: 4bf34117-aadf-400d-91ec-77dc8633874f
Found the id: e3bc023c-27a1-496f-8a5c-665b0e103e60
Found the id: 4e6f8c5e-1b70-4289-a2ab-ae9065304eb5


['a81a99e2-90bd-494c-ae42-7090f93115ca',
 '42c85d23-21e4-4fe9-baef-9ab06c0cbe6d',
 '4bf34117-aadf-400d-91ec-77dc8633874f',
 'e3bc023c-27a1-496f-8a5c-665b0e103e60',
 '4e6f8c5e-1b70-4289-a2ab-ae9065304eb5']

In [5]:
#Get all test centers
test_centers = soup.find_all('div', class_="list__result")

#Getting info about the page header
header = soup.find('div', class_="list__headline").text.strip()
print(f" Header: {header}") 

# Loop getting company names of all test centers
for test_center in test_centers:
    company_name = test_center.find('span', class_="list__col").text.replace('\n ', '')
    #callendar= test_center.find('div', class_="main")#.text.replace('\n ', '')
    print(f" Company Name : {company_name.strip()}")   
    print(' ')

 Header: Hlavní město Praha (Počet OM: 113 - V následujících 3 dnech volná/celková kapacita: 28571 / 34182)
 Company Name : #NaŽádanku.cz – Odběrové centrum Praha 15 (H. Měcholupy)
 
 Company Name : #Vyšetři.mě – Odběrové centrum Praha 15 (H. Měcholupy)
 
 Company Name : #Vyšetři.mě – Odběrové centrum Praha 6 (Ruzyně)
 
 Company Name : AESKULAB - odběrové místo VIVO Hostivař
 
 Company Name : AESKULAB Praha 6 - OM poliklinika DAM, Brixiho
 
 Company Name : AESKULAB Praha 8 - Mazurská
 
 Company Name : AESKULAB Praha Čs.exilu
 
 Company Name : AESKULAB Praha Vajgarská
 
 Company Name : AKESO holding - odběrové místo Praha Eden
 
 Company Name : AKESO holding - odběrové místo Praha Chlumecká
 
 Company Name : AKESO holding - OM Jeremiášova, Praha 5 - Stodůlky
 
 Company Name : AKESO holding - OM Seydlerova, Praha 5 - Nové Butovice (Poliklinika Lípa)
 
 Company Name : AKESO holding - OM Stroupežnického, Praha 5 - Anděl
 
 Company Name : AURUM CLINIC s.r.o. - odběrové místo Zborovská
 
 Co

In [6]:
header = soup.find('div', class_="list__headline").text.strip()
header # in order to check how many free places are available

'Hlavní město Praha (Počet OM: 113 - V následujících 3 dnech volná/celková kapacita: 28571 / 34182)'

In [9]:
# Function returning links for each individual testing center based on url.

def get_page_links(url):# Firstly created the function gets the data and returns the links of the each company 
    sleep(0.2) 
    base_url='https://testovani.uzis.cz/'
    r = requests.get(url)
    soup= BeautifulSoup(r.text,'lxml')
    links=soup.select('div.list__result a')
    return[ base_url + link.attrs['href'] for link in links]

In [10]:
# Function getting necessary details from all companies' pages with Calendar included
# Some information have been commented out due to output format including unnecessary itemization, extra lines, etc.

def test_locatio_detail(url): # Created the second function which grabs all necessary information form each company and returns
    r = requests.get(url)
    soup= BeautifulSoup(r.text,'lxml')
    company_details={
        'Name':soup.select_one('div.detail__main h1').text.strip().replace('\n',' '),
        'Address':soup.select_one('div.info td:nth-child(2)').text.strip().replace('\n',''),
        'Upřesnění polohy':soup.select_one('div.info :nth-child(2) td:nth-child(2)').text.strip().replace('\n',''),
        'Telefon':soup.select_one('div.info a').text.strip().replace('\n',''),
        #'Poznámka':soup.select_one('div.info :nth-child(4) td:nth-child(2)').text.strip().replace('\n','').replace('\n',' '),
        'Příjem':soup.select_one('div.info :nth-child(5) td:nth-child(2)').text.strip().replace('\n','').replace('\n',' '),
        'Cena za samoodběr':soup.select_one('div.info :nth-child(6) td:nth-child(2)').text.strip().replace('\n','').replace('\n',' '),
        'Způsob rezervace':soup.select_one('div.info :nth-child(7) td:nth-child(2)').text.strip().replace('\n','').replace('\n',' '),
        'Odkaz na rezervační systém':soup.select_one('div.info :nth-child(8) td:nth-child(2)').text.strip().replace('\n',''),
        'DRIVE-IN':soup.select_one('div.info :nth-child(9) td:nth-child(2)').text.strip().replace('\n',''),
        'Calendar':soup.select_one('div.slots__list').text.strip().replace('\n',' ')
        #'Av_days':pd.DataFrame([soup.select_one('div.opening').text.strip().replace('\n','')])
   
    }
    #print(company_details)    
    return company_details

#Testing the output of the above mentioned fuction with one defined url
f=test_locatio_detail('https://testovani.uzis.cz//Detail?id=b84211a4-9c9e-4773-8439-71bb18fde2d4&backURL')
pd.DataFrame(f,index=['1',])

,Name,Address,Upřesnění polohy,Telefon,Příjem,Cena za samoodběr,Způsob rezervace,Odkaz na rezervační systém,DRIVE-IN,Calendar
1,"Vsetínská nemocnice, a.s., odběrové místo","Nemocniční 955, 755 01 Vsetín",pavilon K,571818384,Osoby se žádankou i samoplátce,,Rezervační systém Reservatic,nemocnice-vs.cz,Ne,St 8.9. Maximální kapacita Čt 9.9. Max...


In [11]:
# Updated copy of previously defined function

#Function main combines both previously defined fuctions: get_page_links and test_locatio_detail
#get_page_links("https://testovani.uzis.cz/")  

def main():# Created 3rd function which focuses on gattering all the information into one dictionary, to be saved in main function
   #results=[]
   urls= get_page_links('https://testovani.uzis.cz/')
   comp_info=pd.DataFrame([test_locatio_detail(url) for url in tqdm(urls)])
   #results.append(comp_info)
   return comp_info

In [12]:
df=main()# to store the value in df object to utilize the usage

100%|██████████| 393/393 [02:12<00:00,  2.96it/s]


In [13]:
df #see how the data frame look

,Name,Address,Upřesnění polohy,Telefon,Příjem,Cena za samoodběr,Způsob rezervace,Odkaz na rezervační systém,DRIVE-IN,Calendar
0,#NaŽádanku.cz – Odběrové centrum Praha 15 (H. ...,"Park na křižovatce ulic Ravennská a Milánská, ...",Zelené oplocení v parku na křižovatce ulic Rav...,+420774605535,Osoby se žádánkou (od lékaře/KHS),,Přes webové stránky na přesný čas,www.nazadanku.cz,Ne,St 8.9. Maximální kapacita Čt 9.9. Vol...
1,#Vyšetři.mě – Odběrové centrum Praha 15 (H. Mě...,"Park na křižovatce ulic Ravennská a Milánská, ...",Zelené oplocení v parku na křižovatce ulic Rav...,+420771156107,Osoby se žádankou i samoplátce,,Přes webové stránky,www.vysetri.me,Ne,St 8.9. Volná kapacita 4 z 4 Čt 9.9. V...
2,#Vyšetři.mě – Odběrové centrum Praha 6 (Ruzyně),Drnovská 1112/60 161 00 Praha 6 - Ruzyně,Příjezdová cesta zezadu budovy (naproti adrese...,+420771156107,Osoby se žádankou i samoplátce,,Přes webové stránky,www.vysetri.me,Ano,St 8.9. Maximální kapacita Čt 9.9. Vol...
3,AESKULAB - odběrové místo VIVO Hostivař,"Praha 10, Švehlova 1391/32","Odběrové místo - OC VIVO Hostivař, parkoviště",+420800737383,Osoby se žádankou i samoplátce,,www.rezervace.aeskulab.cz,rezervace.aeskulab.cz,Ne,St 8.9. Žádná data Čt 9.9. Žádná data ...
4,"AESKULAB Praha 6 - OM poliklinika DAM, Brixiho","Brixiho 1968/1, 162 00 Praha 6 - Břevnov","Poliklinika DAM, samostatný vchod vpravo z Bri...",+420800737383,Osoby se žádankou i samoplátce,,Přes webové stránky,rezervace.aeskulab.cz,Ne,St 8.9. Žádná data Čt 9.9. Volná kapac...
...,...,...,...,...,...,...,...,...,...,...
388,Nemocnice Valašské Meziříčí - odběrové místo,"U Nemocnice 980, 757 01, Valašské Meziříčí","blok C - 2. patro, uvnitř nemocnice",571758605,Osoby se žádankou i samoplátce,,Pro samoplátce – on-line,www.agellab.cz,Ne,St 8.9. Žádná data Čt 9.9. Žádná data ...
389,"Uherskohradišťská nemocnice a. s., odběrové místo","J. E. Purkyně 365, 686 06 Uherské Hradiště",areál nemocnice,724631022,Osoby se žádankou i samoplátce,,Přes webové stránky,ambulance.nemuh.cz,Ne,St 8.9. Žádná data Čt 9.9. Žádná data ...
390,"Vaše laboratoře s.r.o., Zlín - odběrové místo",Tř. T. Bati 3910,po levé straně budovy,739408931,Osoby se žádánkou (od lékaře/KHS),,Reservatic,reservatic.com,Ne,St 8.9. Volná kapacita 54 z 72 Čt 9.9. ...
391,Vsetínská nemocnice a.s. - pavilon K,"Nemocniční 955, Vsetín 755 01",pavilon K,,Osoby se žádankou i samoplátce,,Rezervační systém Reservatic,nemocnice-vs.cz,Ne,St 8.9. Maximální kapacita Čt 9.9. Vol...


In [15]:
df.to_csv('Covid.csv', encoding='utf-8-sig')# To save the scraped data under "Covid csv"

We will next load the data to our working directory to process data and put it in commercially friendly way

In [28]:
# Reading and displaying data from .csv file
# For a quick data check only, repeated in the processor part as well.

data = pd.read_csv('Covid.csv', index_col = 0)
data.head(5)

,Name,Address,Upřesnění polohy,Telefon,Příjem,Cena za samoodběr,Způsob rezervace,Odkaz na rezervační systém,DRIVE-IN,Calendar
0,#NaŽádanku.cz – Odběrové centrum Praha 15 (H. ...,"Park na křižovatce ulic Ravennská a Milánská, ...",Zelené oplocení v parku na křižovatce ulic Rav...,4.207746e+11,Osoby se žádánkou (od lékaře/KHS),NaN,Přes webové stránky na přesný čas,www.nazadanku.cz,Ne,St 8.9. Maximální kapacita Čt 9.9. Vol...
1,#Vyšetři.mě – Odběrové centrum Praha 15 (H. Mě...,"Park na křižovatce ulic Ravennská a Milánská, ...",Zelené oplocení v parku na křižovatce ulic Rav...,4.207712e+11,Osoby se žádankou i samoplátce,NaN,Přes webové stránky,www.vysetri.me,Ne,St 8.9. Volná kapacita 4 z 4 Čt 9.9. V...
2,#Vyšetři.mě – Odběrové centrum Praha 6 (Ruzyně),Drnovská 1112/60 161 00 Praha 6 - Ruzyně,Příjezdová cesta zezadu budovy (naproti adrese...,4.207712e+11,Osoby se žádankou i samoplátce,NaN,Přes webové stránky,www.vysetri.me,Ano,St 8.9. Maximální kapacita Čt 9.9. Vol...
3,AESKULAB - odběrové místo VIVO Hostivař,"Praha 10, Švehlova 1391/32","Odběrové místo - OC VIVO Hostivař, parkoviště",4.208007e+11,Osoby se žádankou i samoplátce,NaN,www.rezervace.aeskulab.cz,rezervace.aeskulab.cz,Ne,St 8.9. Žádná data Čt 9.9. Žádná data ...
4,"AESKULAB Praha 6 - OM poliklinika DAM, Brixiho","Brixiho 1968/1, 162 00 Praha 6 - Břevnov","Poliklinika DAM, samostatný vchod vpravo z Bri...",4.208007e+11,Osoby se žádankou i samoplátce,NaN,Přes webové stránky,rezervace.aeskulab.cz,Ne,St 8.9. Žádná data Čt 9.9. Volná kapac...


In [30]:
def calnedar(url): # Created the second function which grabs all necessary information form each comapny and returns
    r = requests.get(url)
    soup= BeautifulSoup(r.text,'lxml')
    company_calendars={
        'Calendar':soup.select_one('div.slots__list').text.strip().replace('\n',' ')
    }
    
    return company_calendars
Ca=calnedar('https://testovani.uzis.cz/Detail?id=f4d02b50-1561-4c7e-9cf4-8d85b2d985ea&backURL=/')
Ca

{'Calendar': 'St 8.9.   Maximální kapacita     Čt 9.9.   Volná kapacita 57 z 81     Pá 10.9.   Volná kapacita 55 z 73     So 11.9.   Volná kapacita 89 z 104     Ne 12.9.   Volná kapacita 97 z 104     Po 13.9.   Volná kapacita 100 z 101     Út 14.9.   Volná kapacita 99 z 101'}

In [31]:
def all_clendarInfo():# Here I will source all available calendar info.
   #results=[]
   urls= get_page_links('https://testovani.uzis.cz/')
   calnedars=pd.DataFrame([calnedar(url) for url in urls])
   #results.append(comp_info)
   return calnedars

all_clendarInfo()

,Calendar
0,St 8.9. Maximální kapacita Čt 9.9. Vol...
1,St 8.9. Maximální kapacita Čt 9.9. Vol...
2,St 8.9. Maximální kapacita Čt 9.9. Vol...
3,St 8.9. Žádná data Čt 9.9. Žádná data ...
4,St 8.9. Žádná data Čt 9.9. Volná kapac...
...,...
388,St 8.9. Žádná data Čt 9.9. Žádná data ...
389,St 8.9. Žádná data Čt 9.9. Žádná data ...
390,St 8.9. Volná kapacita 54 z 72 Čt 9.9. ...
391,St 8.9. Maximální kapacita Čt 9.9. Vol...


Getting capacities info from each calendar line


In [35]:
cad_dataframe=all_clendarInfo()
cad_dataframe.to_csv('calendar.csv')